In [1]:
# required Python imports
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.18 imported


## Step 1: Create the database

Questions to be investigated:

- How many measurements are required to obtain the orientation with an error of, say, ±1 degree?
- What range of azimuths should we consider? 90, 180, 360? Does it matter?
- Are there special cases leaving aside the gymbal lock issue (see below)?
- Other considerations?

> ### An important note about the gymbal lock issue
>
> A special case when using Euler angles is the so-called gymbal lock. This occurs when the second Euler angle is at one of its limits (0, 360, or 180 in the case of an orthorhombic system). In this case, the axes of rotation of the first and last Euler rotations are either the same or exactly opposite, i.e. an infinite number of angles $\phi1$ and $\phi2$ will represent the same rotation even if we restrict the range to an interval of length 2π: infinite pairs of $\phi1$ and $\phi2$ angles satisfy
>
> $$
> \phi1 ± \phi2 = \text{constant}
> $$
>
> For example, all these triplets will represent the same orientation (all representing a 45 degree rotation along the z-axis):
>
> $[45, 0, 0], [30, 0, 15], [50, 0, -5], [0, 0, 45], [55, 0, 170]...\text{and so on} $

### 1.1 Euler angles

In [2]:
# check explore Euler space avoiding Phi=0
f.explore_Euler_space(step=1, lower_bounds=(0, 5, 0))

array([[  0,   5,   0],
       [  0,   5,   1],
       [  0,   5,   2],
       ...,
       [ 90,  90, 178],
       [ 90,  90, 179],
       [ 90,  90, 180]])

In [3]:
# STEP 1.1: generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=27, lower_bounds=(0, 9, 0)))
database.euler.shape

(112, 3)

### 1.2 Generate reference T envelope

In [4]:
# STEP 1.2: Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [5]:
# example consider the full range of azimuths and 16 measures
np.arange(0, 360, 45/2)

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5])

In [6]:
# example consider the a 180 degrees range and 16 measures
np.arange(0, 180, 45/4)

array([  0.  ,  11.25,  22.5 ,  33.75,  45.  ,  56.25,  67.5 ,  78.75,
        90.  , 101.25, 112.5 , 123.75, 135.  , 146.25, 157.5 , 168.75])

### 1.3 Calculate and store in the database the T values and azimuths of sections

In [7]:
# STEP 1.3: Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [8]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(112, 3)
(112, 16)
(112, 16)


In [9]:
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 1))
print('Azimuths: ', np.around(database.azimuths[-1], 1))

Euler angles:  [ 81  90 162]
T values:  [48.  37.8 26.3 20.2 20.  20.  20.  20.  20.  20.  20.  20.  22.9 33.1
 44.6 50.8]
Azimuths:  [  0.1  22.6  45.1  66.9  72.   72.   72.   72.  252.  252.  252.  252.
 269.8 292.6 314.9 337.5]


In [10]:
# for i in range(45):
#     print(np.around(database.azimuths[i], 1), '; ', np.around(database.euler[i], 0))

## 2. Test minimization algorithms

In [11]:
for index, orientation in enumerate(database.euler):
    print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    print('')
    print('default algorithm')
    f.find_orientation(measurements=measures, params=(90, 50, 20), silent=False)

    print('differential evolution algorithm')
    f.find_orientation_diffevol(measurements=measures, params=(90, 50, 20), silent=False)

    print('dual annealing algorithm')
    f.find_orientation_annealing(measurements=measures, params=(90, 50, 20), silent=False)
    print('----------------------')
    print('')

Real: [0 9 0]

default algorithm
Calculated Orientation: [  0.   9. 180.]
differential evolution algorithm
Calculated Orientation: [  1.   9. 179.]
dual annealing algorithm
Calculated Orientation: [90.  2. 90.]
----------------------

Real: [ 0  9 27]

default algorithm
Calculated Orientation: [ 1.  9. 26.]
differential evolution algorithm
Calculated Orientation: [ 28.   8. 180.]
dual annealing algorithm
Calculated Orientation: [ 1.  9. 26.]
----------------------

Real: [ 0  9 54]

default algorithm
Calculated Orientation: [ 1.  9. 53.]
differential evolution algorithm
Calculated Orientation: [ 1.  9. 53.]
dual annealing algorithm
Calculated Orientation: [ 1.  9. 53.]
----------------------

Real: [ 0  9 81]

default algorithm
Calculated Orientation: [ 1.  9. 80.]
differential evolution algorithm
Calculated Orientation: [ 1.  9. 80.]
dual annealing algorithm
Calculated Orientation: [ 1.  9. 80.]
----------------------

Real: [  0   9 108]

default algorithm
Calculated Orientation: [  

Let's check now orientations where the gymbal lock happens (also close orientations)

Note that if $\Phi$ is well constrained and $\phi_1$ and $\phi_2$ meet one of the following criteria (det denotes determinedby algorithm):

$$
\phi_{1} + \phi_{2} = \phi_{1\text{det}} + \phi_{2\text{det}} \quad | \quad \phi_{1} + \phi_{2} = \phi_{1\text{det}} - \phi_{2\text{det}} \quad | \quad \phi_{1} + \phi_{2} = \phi_{1\text{det}} + \phi_{2\text{det}} - 180
$$

the orientation is well constrained.

In [12]:
phi1 = np.arange(0, 91, 30)
Phi = np.arange(0, 5, 1)
phi2 = np.arange(0, 181, 60)

# Create a meshgrid of all possible combinations
phi1, Phi, phi2 = np.meshgrid(phi1, Phi, phi2, indexing='ij')
# Stack the angles along the third axis to create the final array
arr = np.stack((phi1, Phi, phi2), axis=-1)
arr = arr.reshape(-1, 3)

database = SimpleNamespace(euler=arr)
database.euler.shape, database.euler[-1]

((80, 3), array([ 90,   4, 180]))

In [13]:
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [14]:
for index, orientation in enumerate(database.euler):
    print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    print('')
    print('default algorithm')
    f.find_orientation(measurements=measures, params=(90, 50, 20), silent=False)

    print('differential evolution algorithm')
    f.find_orientation_diffevol(measurements=measures, params=(90, 50, 20), silent=False)

    print('dual annealing algorithm')
    f.find_orientation_annealing(measurements=measures, params=(90, 50, 20), silent=False)
    print('----------------------')
    print('')

Real: [0 0 0]

default algorithm
Calculated Orientation: [ 44.   0. 136.]
differential evolution algorithm
Calculated Orientation: [ 43.   0. 137.]
dual annealing algorithm
Calculated Orientation: [ 70.   0. 110.]
----------------------

Real: [ 0  0 60]

default algorithm
Calculated Orientation: [34.  0. 26.]
differential evolution algorithm
Calculated Orientation: [43.  0. 17.]
dual annealing algorithm
Calculated Orientation: [13.  0. 47.]
----------------------

Real: [  0   0 120]

default algorithm
Calculated Orientation: [23.  0. 97.]
differential evolution algorithm
Calculated Orientation: [43.  0. 77.]
dual annealing algorithm
Calculated Orientation: [ 13.   0. 107.]
----------------------

Real: [  0   0 180]

default algorithm
Calculated Orientation: [ 42.   0. 138.]
differential evolution algorithm
Calculated Orientation: [ 43.   0. 137.]
dual annealing algorithm
Calculated Orientation: [81.  0. 99.]
----------------------

Real: [0 1 0]

default algorithm
Calculated Orienta

As can be seen, $\Phi$ is well determined with a variation of ± 1 degree while $\phi_1$ and $\phi_2$ meet the criteria indicated above. So, orientations are well constrained. However, it seems that the gymbal lock is not only present at $\Phi$ = 0, but also at angles approaching zero (there are cases even at $\Phi$ = 4).

TODO:
- Carry out a test comparing the different methods at the same time (time, accuracy, etc.).
- The question of range of azimuths and number of points vs. accuracy.


In [15]:
import sys
from datetime import date    
today = date.today().isoformat()

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)

Notebook tested in 2024-03-18 using:
Python 3.11.8 | packaged by Anaconda, Inc. | (main, Feb 26 2024, 21:34:05) [MSC v.1916 64 bit (AMD64)]
Numpy 1.26.4
